# Lesson 4: Creating a cell counting module in PyCharm

Today we will be building a module in Python from scratch. The goal is to have this module do a little bit of histology analysis. It won't produce publication-ready figures, but can at least show you the beginnings of what you can accomplish using Python. 

The first step before writing any code at all is to think about the algorithm behind the intended analysis. At technical interviews for software engineering positions, interviewers ask for code that will solve a particular problem that they present. Before you even touch the keyboard, you must first think about the problem and the steps that you need to code up to solve the problem. Then you communicate your thought process to the interviewer. It is imperative that you understand what the problem is before you try to solve it. 

Today we will write some code to plot some histology images with circled cell bodies. 

**What are the steps required to accomplish this? List them in order.**

# Step 0: Create new Python module

Before you start coding, you will need to create a .py file to open in the PyCharm editor. 

1. Open PyCharm and your PyCharm project. Your project should automatically load when you start up PyCharm, but if not, go to File -> Open -> navigate to PythonForNeuro folder -> Ok.
2. On the menu to the left, right click PythonForNeuro folder -> New -> Python File -> Type in an appropriate name (e.g., histology) -> Ok. 

The typical anatomy of a Python module is as follows in this specific order: 

1. Import relevant outside packages (e.g., `numpy`).
2. All the code that you develop (e.g., functions). 
3. A `__name__` block to make debugging easier. 

Step 3 may not make sense right now, but we will get to it soon. 

# Step 1: Load .czi file

The first thing we want to do is to load the contents of the .czi file. At the beinning of your Python module, _make sure to import the necessary package(s)_. Then, define a function that takes an input `fname`, the name of the .czi file. Name it something informative (e.g., `load_frame`). 

In the body of the function definition, enter the following: 

In [ ]:
with CziFile(fname) as czi:
    im = czi.asarray()

This code block will open the .czi file, temporarily store its contents in `czi`, then convert that convert that content into an array. Next, enter code that will return the array and exit the function. 

# Step 1.1: Debug load function

Cool, now how can we check whether this works? This is where debugging comes in. Debugging is very useful for examining checkpoints and checking whether code is doing what you think it's doing. That requires going inside the processes and functions that you've defined and inspecting variables to see whether they match your expectations. Let's set up the `__name__` block.

The `__name__` block looks like this: 

In [ ]:
if __name__ == '__main__':
    #CODE HERE

For now, use this: 

In [ ]:
if __name__ == '__main__':
    fname = #file name here
    
    frame = load_frame(fname)

The details behind why this is necessary are not critical, but if you are curious, read this: https://stackoverflow.com/questions/419163/what-does-if-name-main-do. Basically, whenever you debug the file, Python will run everything above the `__name__` block (which will be `import` statements and function definitions). Then the code in the `__name__` block will call those functions (e.g., `load_frame`). 

To debug, first insert a _breakpoint_ in `load_frame` by clicking the space in between the line number and the actual code. Insert this breakpoint on the line that returns your frame in the `load_frame` function. A red dot should appear on that line. Now, when you enter debug mode, the `__name__` block will run and the code will stop before the breakpoint. Try doing this by right clicking on the editor and clicking `Debug 'histology'`. 

You should see the line with the breakpoint highlighted. 
1. Click on the Debugger tab and you will see the variables in the workspace. 
2. Inspect `im`. What is its data type? What are its dimensions? Is this expected? 
3. You can click `View as Array` to look at its values. 


# Step 1.2: Cleaning and smoothing data

For this step, we will:
1. Get rid of the singleton dimensions in `im`. 
2. Extract a specific frame from `im`. 
3. Smooth that frame.

At the same time, we will explore the debugging environment (note that I'm using the word environment to mean something different from a conda env). 

`im` has a lot of singleton dimensions which don't contain any data. 
1. Use the `numpy` function `squeeze` to remove these dimensions and add this code to your `load_frame` definition. 
2. Place a breakpoint on this line, remove the previous breakpoint, then enter the debugger again (you can click the green circular arrow to the left of the Debugger tab). 
3. You are now stopped right before `squeeze` happens. Inspect `im`. 
4. Then click the bent blue arrow (`Step Over`). This causes Python to run the next line (which should be the `squeeze` call) and stop. 
5. Inspect the output of your `squeeze`. How is it different from the original `im`? 

Let's define our `load_frame` function to also select a channel as well as a z position. Add these as arguments into your definition. Our squeezed `im` should have the shape `(3, 13, 1946, 1946)`. This corresponds to (channel, z position, x position, y position) in our array of z stacks. Enter a line of code that takes a frame from a channel and a z position specified by the input arguments. I recommend starting on `z=10` and `channel=1`. 

Click the calculator-like button (`Evaluate Expression`). This opens up a window that lets you enter code inside the debugging environment. Use this to plot the frame with `plt.imshow()`. 

Now, smooth your image. Use the function `gaussian_filter` from `scipy.ndimage.filters`. Refer to this reference: https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.filters.gaussian_filter.html. I recommend using a `sigma` of 3 while keeping the other parameters default. 

At any point in this process, restart the debugger and insert breakpoints whenever you want to inspect variables and see whether your function is performing as intended. Change stuff and see what happens. At the end of this step, you should return a `(1946, 1946)` array. 

# Step 2: Detect blobs (cells)

Now we can look for our cells. Luckily, blob detection code already exists. Define a new function that runs `blob_log` from the `skimage.feature` package and call it `detect_blobs`. It should return the output of `blob_log`. `detect_blobs` should take parameters `threshold`, `min_sigma`, and `max_sigma` that will be passed into `blob_log`. For defaults, I recommend `threshold=0.04`, `min_sigma=10`, `max_sigma=30`. Refer to this documentation for more information: https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_log.

# Step 3: Plot blobs